## Fundamentação Comportamental e Feature Engineering

#### Inteligência Preditiva para Intervenção de Crédito Responsável

Justificativa da necessidade do modelo de ML e Dicionário de Features baseadas no risco brasileiro, **validado por resultados Out-of-Time (OOT)**.

---

### 1. Introdução: O Ciclo Vicioso e a Tese da Intervenção Proativa

**Insight Validado:** O modelo confirmou que **exposição total ao crédito (limite)** e **risco sistêmico (SCR)** são os principais preditores, superando até mesmo a utilização percentual do limite.

**Estratégia Otimizada:** Threshold em **0.20** garantindo **100% de recall** para capturar todos os clientes em risco real, transformando falsos positivos em oportunidades de fidelização.

---


## 2. 🇧🇷 Panorama do Endividamento e a Urgência Preditiva (Contexto de Mercado)

O sucesso deste modelo preditivo é diretamente proporcional à escalada do endividamento no Brasil, que move clientes historicamente bons para o alto risco de crédito (Rotativo).

### 2.1. O Estágio do Risco no Consumidor (Serasa/ANBIMA)

A tese de *Feature Engineering* é solidamente ancorada em dados de mercado que demonstram a magnitude e a fragilidade financeira do consumidor brasileiro:

| Fonte | Dado Relevante (SETEMBRO/2025) | Implicação para o Negócio |
| :--- | :--- | :--- |
| **SERASA** | **79,1 milhões de inadimplentes** e valor médio da dívida de **R$ 6.274,82** por pessoa. | **Justifica a urgência:** Demonstra a magnitude do risco sistêmico. A intervenção proativa se torna uma prioridade de capital e risco. |
| **ANBIMA** | O **"Raio X do Investidor Brasileiro"** é a principal referência sobre o tema, validando a necessidade de uma abordagem **comportamental** e não apenas estática na gestão de risco. | **Legitima a Tese:** Endossa a estratégia de focar em tendências de gasto e comprometimento implícito como sinais reais de estresse. |

### 2.2. A Tese do Estresse Agudo e as Features Comportamentais Chave

Nossa estratégia transforma esse contexto macro em variáveis preditivas. As *features* mais críticas são construídas especificamente para capturar os **sinais precoces de esgotamento de liquidez** que antecedem a entrada no Rotativo, garantindo que o modelo aprenda com o comportamento de crise:

* **`scr_tcr_implicita` (Taxa de Comprometimento Implícita):**
    * **Lógica:** Esta variável é a principal *proxy* para medir a **pressão financeira real**. Captura o aumento da dependência do cliente em usar o limite disponível para cobrir despesas básicas ou outras dívidas, um **sinal clássico de estresse** que precede o Rotativo. Sua importância elevada no modelo **valida a tese comportamental**.
* **`gasto_crescim_3m` (Crescimento do Gasto em 3 Meses):**
    * **Lógica:** Monitora o **desalinhamento orçamentário**. O crescimento do gasto descontrolado é um indicador de que o cliente está perdendo a capacidade de fechar as contas apenas com a renda, forçando o uso de crédito de alto custo. A inclusão de uma análise *lagged* (tendencial) é crucial aqui para evitar *data leakage* e garantir o poder preditivo.

---

### 3. O Foco no Estresse Agudo: Cartão de Crédito e Sinais Dinâmicos

**Objetivo:** Isolar as linhas de crédito que são o alvo direto da intervenção e as features que capturam a mudança de comportamento.

| Variável BACEN | Fonte (API) | Conclusão para o Projeto |
| :--- | :--- | :--- |
| Concessões de crédito - Cheque especial | bcdata.sgs.20665 | O Cheque Especial e o Rotativo são os principais gatilhos de estresse. A Feature Engineering foca em **médias móveis e taxas de crescimento** para capturar o uso recorrente e a aceleração do endividamento. |
| Concessões de crédito - Cartão de crédito rotativo | bcdata.sgs.20679 | O uso do Rotativo é um forte preditor. Nossa intervenção permite à Fintech oferecer a **liquidação do Rotativo com Empréstimo Pessoal** antes da Resolução BACEN 4.549 obrigar o parcelamento (uma solução pior para o cliente). |


---

### 4. Fatores de Risco Comportamental (Validados pelo Modelo Final)

**Objetivo:** Justificar a inclusão de features que foram comprovadamente os maiores preditores do risco no modelo LightGBM.

| Risco Comportamental (Teoria) | Implicação para a Feature Engineering | Validação (Importância do Modelo) |
| :--- | :--- | :--- |
| Exposição ao Crédito | O limite total do cartão é o fator mais crítico de risco | Feature #1: limite_cartao (354 de importância) |
| Risco Sistêmico (SCR) | Exposição total no sistema financeiro através do TCR | Feature #2: scr_tcr_implicita (333 de importância) |
| Estresse de Liquidez Imediata | Utilização recorrente do limite como proxy de dependência	| Feature #3: utilizacao_limite_media_3m (297 de importância) |
| Deterioração Financeira | Crescimento acelerado da dívida total no mercado | Feature #4: scr_crescim_divida_3m (208 de importância) |
| Estresse de Consumo | Aceleração do gasto mensal precedendo incapacidade de pagamento | Feature #5: gasto_crescim_3m (186 de importância) |
| Comportamento Rotativo | Frequência de uso do crédito rotativo | Feature #6: contagem_rotativo_3m (40 de importância) |

---

### 5. Feature Set Final: Dicionário de Variáveis e Validação Preditiva

O conjunto de features foi desenhado em três pilares para garantir a máxima predição do Estresse Agudo, conforme a **Validação Out-of-Time (OOT)** do modelo LightGBM:

#### 5.1. Pilares e Features Otimizadas

| Categoria | Feature | Justificativa e Importância Comprovada |
| :--- | :--- | :--- |
| **Pilar 1: Exposição ao Crédito** | `limite_cartao` | **#1 no ranking (354)** Fator mais crítico de risco | 
| **Pilar 2: Risco Sistêmico (SCR)** | `scr_tcr_implicita` | **#2 no ranking (333)** Exposição total no sistema financeiro |
| **Pilar 3: Estresse de Liquidez** | `utilizacao_limite_media_3m` | **#3 no ranking (297)** Utilização recorrente do limite |
| **Pilar 2: Risco Sistêmico (SCR)** | `scr_crescim_divida_3m` | **#4 no ranking (208)** Crescimento acelerado da dívida |
| **Pilar 3: Estresse de Consumo** | `gasto_crescim_3m` | **#5 no ranking (186)** Aceleração do gasto mensal |
| **Pilar 4: Comportamento Rotativo** | `contagem_rotativo_3m` | **#6 no ranking (40)** Frequência de uso do rotativo |

---

### 6. Conclusão Estratégica: Performance Comprovada e Implantação

#### 6.1. O Ciclo se Concretizou:

1. **Fundamentação Teórica** → Hipótese de estresse agudo validada com novos insights sobre importância do limite absoluto
2. **Modelo Preditivo (LightGBM)** validado com performance robusta e de alta diferenciação (AUC = 0.7776) em Validação Out-of-Time (OOT)
3. **Estratégia Otimizada** → Threshold ajustado para **0.20**, resultando em Recall de **100%** para a Classe 1 (Rotativo)
4. **Viabilidade Operacional** → O Trade-off (Precision 36%) é gerenciado transformando Falsos Positivos em **Oportunidades de Fidelização** através da intervenção consultiva


#### 6.2. Resultados de Performance (OOT - Out-of-Time):

- **AUC Score:** 0.7721
- **Recall (Classe 1):** 100%
- **Precision (Classe 1):** 36%
- **F1-Score (Classe 1):** 53%
- **Acurácia Global:** 55%
- **Distribuição do Target:** 79.11% Classe 0 vs 20.89% Classe 1
- **Peso Otimizado da Classe 1:** 4.19x

#### 6.3. Insights Estratégicos Descobertos:

Surpresas no Ranking de Features:

1. Limite do Cartão (#1): Exposto como preditor mais importante, redefinindo prioridades de monitoramento

2. SCR TCR Implícita (#2): Risco sistêmico superou expectativas como indicador de estresse

3. Features Demográficas: Baixa importância relativa confirmando foco comportamental sobre sociodemográfico

Validação do Desbalanceamento:

1. Distribuição natural de 79.11% vs 20.89% reflete realidade operacional

2. Estratégia de peso (4.19x) comprovadamente eficaz para recall máximo

#### 6.4. Próximos Passos para Implantação:

1. Integração com Sistemas de cobrança e atendimento ao cliente

2. Automação de Alertas para equipe de relacionamento baseada no novo ranking de features

3. Dashboard de Monitoramento contínuo da performance do modelo com foco nas top 5 features

4. Retreinamento Periódico com dados atualizados do mercado

5. Revisão de Políticas de limite considerando a descoberta sobre importância do limite absoluto

### 7. . Análise de Impacto Business

Eficácia Comprovada: O modelo captura 100% dos clientes em risco real de entrar no rotativo, permitindo intervenção proativa.

Oportunidade de Negócio: Os 64% de falsos positivos representam clientes de baixo risco que recebem ofertas de empréstimo personalizado, aumentando cross-selling e fidelização.

Diferencial Competitivo: Manter inadimplência abaixo da média do mercado através de inteligência preditiva validada estatisticamente.